Deep Learning
=============

Assignment 6
------------

After training a skip-gram model in `5_word2vec.ipynb`, the goal of this notebook is to train a LSTM character model over [Text8](http://mattmahoney.net/dc/textdata) data.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import os
import numpy as np
import random
import string
import tensorflow as tf
import zipfile
from six.moves import range
from six.moves.urllib.request import urlretrieve
import time

In [2]:
url = 'http://mattmahoney.net/dc/'

def maybe_download(filename, expected_bytes):
  """Download a file if not present, and make sure it's the right size."""
  if not os.path.exists(filename):
    filename, _ = urlretrieve(url + filename, filename)
  statinfo = os.stat(filename)
  if statinfo.st_size == expected_bytes:
    print('Found and verified %s.' % filename)
  else:
    print(statinfo.st_size)
    raise Exception(
      'Failed to verify ' + filename + '. Can you get to it with a browser?')
  return filename

filename = maybe_download('text8.zip', 31344016)

Found and verified text8.zip.


In [3]:
def read_data(filename):
  f = zipfile.ZipFile(filename)
  for name in f.namelist():
    return tf.compat.as_str(f.read(name))
  f.close()
  
text = read_data(filename)
print('Data size %d.' % len(text))

Data size 100000000.


Create a small validation set.

In [4]:
valid_size = 1000
valid_text = text[:valid_size]
train_text = text[valid_size:]
train_size = len(train_text)
print(train_size, train_text[:64])
print(valid_size, valid_text[:64])

99999000 ons anarchists advocate social relations based upon voluntary as
1000  anarchism originated as a term of abuse first used against earl


Utility functions to map characters to vocabulary IDs and back.

In [5]:
vocabulary_size = len(string.ascii_lowercase) + 1 # [a-z] + ' ' = 27
first_letter = ord(string.ascii_lowercase[0])

def char2id(char):
  if char in string.ascii_lowercase:
    return ord(char) - first_letter + 1
  elif char == ' ':
    return 0
  else:
    print('Unexpected character: %s' % char)
    return 0
  
def id2char(dictid):
  if dictid > 0:
    return chr(dictid + first_letter - 1)
  else:
    return ' '

print(char2id('a'), char2id('z'), char2id(' '), char2id('ï'))
print(id2char(1), id2char(26), id2char(0))

Unexpected character: ï
1 26 0 0
a z  


Function to generate a training batch for the LSTM model. **_I don't know how the batches are used yet._**

Note that batch, whose shape is (batch_size, 27), is not continuous in text order. 
BatchGenerator::next() function returns (1+num_unrollings) batches.

batches2string() takes in batches parameter gernetated by BatchGenerator::next() and produces a list of batch_size strings . Each string in the returned list is restored to the text sequence of (1+num_unrollings) charactors.  

In [6]:
batch_size=64
num_unrollings=10

class BatchGenerator(object):
  def __init__(self, text, batch_size, num_unrollings):
    self._text = text
    self._text_size = len(text)
    self._batch_size = batch_size
    self._num_unrollings = num_unrollings
    segment = self._text_size // batch_size
    # index into text. distance between successive index are segment
    self._cursor = [ offset * segment for offset in range(batch_size)] 
    self._last_batch = self._next_batch()
  
  def _next_batch(self):  # A batch (whose shape is (batch_size, 27)) is not continuous in text order.
    """Generate a single batch from the current cursor position in the data."""
    batch = np.zeros(shape=(self._batch_size, vocabulary_size), dtype=np.float) # 'batch' shape is (batch_size, 27).
    for b in range(self._batch_size):
      batch[b, char2id(self._text[self._cursor[b]])] = 1.0
      self._cursor[b] = (self._cursor[b] + 1) % self._text_size
    return batch
  
  def next(self):  # The function returns 1+num_unrollings batches.
    """Generate the next array of batches from the data. The array consists of
    the last batch of the previous array, followed by num_unrollings new ones.
    """
    batches = [self._last_batch]
    for step in range(self._num_unrollings):  # Skip num_unrollings batches
      batches.append(self._next_batch())
    self._last_batch = batches[-1]
    return batches

def characters(probabilities):
  """Turn a 1-hot encoding or a probability distribution over the possible
  characters back into its (most likely) character representation."""
  return [id2char(c) for c in np.argmax(probabilities, 1)]  # np.argmax() returns the indices of the maximum values along an axis.

def batches2string(batches):
  """Convert a sequence of batches back into their (most likely) string
  representation."""
  #print(batches[0].shape)
  s = [''] * batches[0].shape[0]
  for b in batches:
    s = [''.join(x) for x in zip(s, characters(b))]  # This line restores text order in element of s
  return s

train_batches = BatchGenerator(train_text, batch_size, num_unrollings)
valid_batches = BatchGenerator(valid_text, 1, 1)

batches = train_batches.next()
print("shape of train batches:", len(batches), batches[0].shape)
print(batches2string(batches))
print(batches2string(train_batches.next()))

batches = valid_batches.next()
print("shape of validation batches:", len(batches), batches[0].shape)
print(batches2string(batches))
print(batches2string(valid_batches.next()))

shape of train batches: 11 (64, 27)
['ons anarchi', 'when milita', 'lleria arch', ' abbeys and', 'married urr', 'hel and ric', 'y and litur', 'ay opened f', 'tion from t', 'migration t', 'new york ot', 'he boeing s', 'e listed wi', 'eber has pr', 'o be made t', 'yer who rec', 'ore signifi', 'a fierce cr', ' two six ei', 'aristotle s', 'ity can be ', ' and intrac', 'tion of the', 'dy to pass ', 'f certain d', 'at it will ', 'e convince ', 'ent told hi', 'ampaign and', 'rver side s', 'ious texts ', 'o capitaliz', 'a duplicate', 'gh ann es d', 'ine january', 'ross zero t', 'cal theorie', 'ast instanc', ' dimensiona', 'most holy m', 't s support', 'u is still ', 'e oscillati', 'o eight sub', 'of italy la', 's the tower', 'klahoma pre', 'erprise lin', 'ws becomes ', 'et in a naz', 'the fabian ', 'etchy to re', ' sharman ne', 'ised empero', 'ting in pol', 'd neo latin', 'th risky ri', 'encyclopedi', 'fense the a', 'duating fro', 'treet grid ', 'ations more', 'appeal of d', 'si have mad']
['i

In [7]:
def logprob(predictions, labels):
  """Log-probability of the true labels in a predicted batch."""
  predictions[predictions < 1e-10] = 1e-10
  # np.log(): Natural logarithm, element-wise.
  # np.sum(): Sum of array elements over a given axis.The default, axis=None, will sum all of the elements of the input array.
  return np.sum(np.multiply(labels, -np.log(predictions))) / labels.shape[0]

def sample_distribution(distribution):
  """Sample one element from a distribution assumed to be an array of normalized
  probabilities.
  """
  # random.uniform() returns a random floating point number N such that a <= N <= b for a <= b
  r = random.uniform(0, 1)
  s = 0
  for i in range(len(distribution)):
    s += distribution[i]
    if s >= r:
      return i
  return len(distribution) - 1

def sample(prediction):
  """Turn a (column) prediction into 1-hot encoded samples."""
  p = np.zeros(shape=[1, vocabulary_size], dtype=np.float)
  p[0, sample_distribution(prediction[0])] = 1.0
  return p

def random_distribution(): # Generate a row of random probabilities
  """Generate a random column of probabilities."""
  b = np.random.uniform(0.0, 1.0, size=[1, vocabulary_size])
  #print(np.sum(b, 1))
  return b/np.sum(b, 1)[:,None]

rd = random_distribution()
print(rd)
print(np.sum(rd))


[[ 0.03958723  0.02493046  0.03579912  0.00954669  0.03519959  0.03003685
   0.03109616  0.05432561  0.06146355  0.01777023  0.00322914  0.0585987
   0.01875511  0.02128574  0.04456404  0.02069084  0.00808751  0.04144652
   0.05933591  0.04093536  0.06058309  0.07152606  0.02819919  0.05582104
   0.0575733   0.0539177   0.01569526]]
1.0


Simple LSTM Model.

In [25]:
#num_nodes = 64
num_nodes = 32

graph = tf.Graph()
with graph.as_default():
  
  # Parameters:
  # Input gate: input, previous output, and bias.
  ix = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  im = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  ib = tf.Variable(tf.zeros([1, num_nodes]))
  # Forget gate: input, previous output, and bias.
  fx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  fm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  fb = tf.Variable(tf.zeros([1, num_nodes]))
  # Memory cell: input, state and bias.                             
  cx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  cm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  cb = tf.Variable(tf.zeros([1, num_nodes]))
  # Output gate: input, previous output, and bias.
  ox = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  om = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  ob = tf.Variable(tf.zeros([1, num_nodes]))
  # Variables saving state across unrollings.
  saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  # Classifier weights and biases.
  w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
  b = tf.Variable(tf.zeros([vocabulary_size]))
  
  # Definition of the cell computation. See Fig 10.6 in P373 and Fig 10.16 on P398 for this model.
  def lstm_cell(i, o, state):
    """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
    Note that in this formulation, we omit the various connections between the
    previous state and the gates."""
    # tf.sigmoid() is an element wise sigmoid function. 
    # These gate's variables have a shape of (batch_size, num_nodes).
    # (batch_size,vocabulary_size)* (vocabulary_size, num_nodes) = (batch_size, num_nodes)
    input_gate = tf.sigmoid(tf.matmul(i, ix) + tf.matmul(o, im) + ib)  
    forget_gate = tf.sigmoid(tf.matmul(i, fx) + tf.matmul(o, fm) + fb)
    # update shape: (batch_size,vocabulary_size)* (vocabulary_size, num_nodes) = (batch_size, num_nodes)
    update = tf.matmul(i, cx) + tf.matmul(o, cm) + cb
    # state shape: (batch_size, num_nodes) * (batch_size, num_nodes) = (batch_size, num_nodes). 
    # Note this is an element wise multiplication.
    state = forget_gate * state + input_gate * tf.tanh(update)
    output_gate = tf.sigmoid(tf.matmul(i, ox) + tf.matmul(o, om) + ob)
    return output_gate * tf.tanh(state), state, tf.shape(forget_gate), tf.shape(update)

  # Input data.
  train_data = list() # create an empty list
  # train_data is a list with (num_unrollings + 1) items each of which is a [batch_size,vocabulary_size] matrix.
  # I.e. train_data shape is (num_unrollings+1, batch_size, vocabulary_size)
  for _ in range(num_unrollings + 1):
    train_data.append(tf.placeholder(tf.float32, shape=[batch_size,vocabulary_size]))
  # Note that train_inputs and train_labels have the same number of elements: num_unrollings
  train_inputs = train_data[:num_unrollings] # all train_data are train_inputs except the last item.
  train_labels = train_data[1:]  # labels are inputs shifted by one time step.

  # Unrolled LSTM loop.
  outputs = list()
  output = saved_output # Shape is (batch_size, num_nodes), initial value are all 0s.
  state = saved_state   # Shape is (batch_size, num_nodes), initial value are all 0s.
  for i in train_inputs:  # i is a (64, 27) matrix, i.e. (batch_size,vocabulary_size)
    # output and state are (batch_size, num_nodes) matrix
    output, state, gate_shape, update_shape = lstm_cell(i, output, state)
    outputs.append(output)

  # Check variable shape
  input_shape=tf.shape(train_inputs)  #(10, 64, 27)
  output_shape=tf.shape(outputs)      #(10, 64, 32)
  state_shape=tf.shape(state)         #(64, 32)

  # outputs are output of num_nodes LSTM cells, with output of num_unrollings times. 
  # State saving across unrollings.
  # How does TF insure loss is calculated after all outputs elements are calculated?
  # Does it make sure all 11 loops to calculate Outputs are execuated before calculating logits?
  # logits are calculated after all outputs elements are collected.
  with tf.control_dependencies([saved_output.assign(output),
                                saved_state.assign(state)]):
    # Classifier.
    logits = tf.nn.xw_plus_b(tf.concat(outputs,0), w, b) 
        # outputs is a list of num_unrollings elements each of which has shape(batch_size, num_nodes).
        # tf.concat(outputs, 0) shape is (num_unrollings*batch_size, num_nodes) = (640, 32)
        # Thus logits shape is (num_unrollings*batch_size, vocabulary_size) = (640, 27)
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(
        logits=logits, labels=tf.concat(train_labels,0)))

  # Optimizer.
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(10.0, global_step, 5000, 0.1, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
  gradients, v = zip(*optimizer.compute_gradients(loss))
  gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
  optimizer = optimizer.apply_gradients(zip(gradients, v), global_step=global_step)

  # Predictions.
  train_prediction = tf.nn.softmax(logits)
  
  # Sampling and validation eval: batch 1, no unrolling.
  sample_input = tf.placeholder(tf.float32, shape=[1, vocabulary_size])
  # The following two variables are not parametered with "trainable=False" becasue no gradient descent 
  # is performed on them.
  saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
  saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
  reset_sample_state = tf.group(
    saved_sample_output.assign(tf.zeros([1, num_nodes])),
    saved_sample_state.assign(tf.zeros([1, num_nodes])))
  sample_output, sample_state, _, _ = lstm_cell(sample_input, saved_sample_output, saved_sample_state)
  with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                saved_sample_state.assign(sample_state)]):
    sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))

In [26]:
num_steps = 3001
summary_frequency = 100

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  mean_loss = 0
  for step in range(num_steps):
    batches = train_batches.next()
    feed_dict = dict()
    for i in range(num_unrollings + 1):
      feed_dict[train_data[i]] = batches[i]
    _, l, predictions, lr, in_shape, out_shape, gSahpe, sShape, uShape = session.run(
      [optimizer, loss, train_prediction, learning_rate, input_shape, output_shape, gate_shape, state_shape,update_shape], feed_dict=feed_dict)
    mean_loss += l
    if step % summary_frequency == 0:
      if step > 0:
        mean_loss = mean_loss / summary_frequency
      # The mean loss is an estimate of the loss over the last few batches.
      print(
        'Average loss at step %d: %f learning rate: %f' % (step, mean_loss, lr))
      mean_loss = 0
      labels = np.concatenate(list(batches)[1:])
      print('Minibatch perplexity: %.2f' % float(
        np.exp(logprob(predictions, labels))))
      if step % (summary_frequency * 10) == 0:
        # Generate some samples.
        print('=' * 80)
        print('input_shape is', in_shape, '.ouputs_shape is', out_shape, '.gate_shape is', gSahpe, '.state_shape is', sShape, '.update_shape is', uShape)
        for _ in range(5):
          feed = sample(random_distribution())
          sentence = characters(feed)[0]
          reset_sample_state.run()
          for _ in range(79):
            prediction = sample_prediction.eval({sample_input: feed})
            feed = sample(prediction)
            sentence += characters(feed)[0]
          print(sentence)
        print('=' * 80)
      # Measure validation set perplexity.
      reset_sample_state.run()
      valid_logprob = 0
      for _ in range(valid_size):
        b = valid_batches.next()
        predictions = sample_prediction.eval({sample_input: b[0]})
        valid_logprob = valid_logprob + logprob(predictions, b[1])
      print('Validation set perplexity: %.2f' % float(np.exp(
        valid_logprob / valid_size)))

Initialized
Average loss at step 0: 3.295903 learning rate: 10.000000
Minibatch perplexity: 27.00
input_shape is [10 64 27] .ouputs_shape is [10 64 32] .gate_shape is [64 32] .state_shape is [64 32] .update_shape is [64 32]
kjtizzrzcws qamhiooaoectwfidt qjpnv  sgiynnfuzhsrhsy nycdattdauzzcantb mqltapekl
nbwfut ypxyaeixtblttwtpeuumjcfa  fls qrqdw vnjiehadj  jlezifdnluecvsnuua ggz by 
heeso nyyeydg tepiahyvdfknpekhf zuiqkkiehkey  gk ktfptkx atihnjmaygsvaajctnu  lp
cqnwluywqkttus grhorer ay m oefkeeruyai jukajochakanbv suqniobtelpgaebejt eufvnj
dtsjroz wv dvncfn udrecqk u oaxh wne ie ijicallblgvzyetbidic ipn twambebmzijvdi 
Validation set perplexity: 20.48
Average loss at step 100: 2.620111 learning rate: 10.000000
Minibatch perplexity: 11.49
Validation set perplexity: 10.69
Average loss at step 200: 2.266261 learning rate: 10.000000
Minibatch perplexity: 9.26
Validation set perplexity: 9.61
Average loss at step 300: 2.108087 learning rate: 10.000000
Minibatch perplexity: 8.57
Validation 

---
Problem 1
---------

You might have noticed that the definition of the LSTM cell involves 4 matrix multiplications with the input, and 4 matrix multiplications with the output. Simplify the expression by using a single matrix multiply for each, and variables that are 4 times larger.

---

---
Problem 2
---------

We want to train a LSTM over bigrams, that is pairs of consecutive characters like 'ab' instead of single characters like 'a'. Since the number of possible bigrams is large, feeding them directly to the LSTM using 1-hot encodings will lead to a very sparse representation that is very wasteful computationally.

a- Introduce an embedding lookup on the inputs, and feed the embeddings to the LSTM cell instead of the inputs themselves.

b- Write a bigram-based LSTM, modeled on the character LSTM above.

c- Introduce Dropout. For best practices on how to use Dropout in LSTMs, refer to this [article](http://arxiv.org/abs/1409.2329).

---

---
Problem 3
---------

(difficult!)

Write a sequence-to-sequence LSTM which mirrors all the words in a sentence. For example, if your input is:

    the quick brown fox
    
the model should attempt to output:

    eht kciuq nworb xof
    
Refer to the lecture on how to put together a sequence-to-sequence model, as well as [this article](http://arxiv.org/abs/1409.3215) for best practices.

---